In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


1 subject.
No feature extraction.

In [3]:
    file = "s01.dat"
    subject_id = 1
    with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

# Random Forest

In [4]:
    clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
    clf.fit(xTrain, np.ravel(yTrain))
    predict = clf.predict(xTest)


    rf_acc = accuracy_score(yTest, predict)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    4.9s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:   11.8s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


# SVM

In [5]:
svm_clf = svm.SVC(kernel='poly', degree=15, C=100, decision_function_shape='ovo')
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)
svm_acc = accuracy_score(yTest, svm_predict)

# kNN

In [ ]:
     n_neighbors = 7

     knn = KNeighborsClassifier(n_neighbors)
     knn.fit(xTrain, np.ravel(yTrain))
     knn_predict = knn.predict(xTest)
     knn_acc = accuracy_score(yTest, knn_predict)

In [ ]:
from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")

NameError: name 'yTest' is not defined

In [ ]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1.csv", mode="a", header=col_names, index=False)